# Approfondiamo l'algoritmo Support Vector Machines
Abbiamo detto che le Support Vector Machines (SVM) sono una classe particolarmente potente e flessibile di algoritmi supervisionati sia per la classificazione, in questi esempi svilupperemo l'intuizione dietro le macchine a vettori di supporto e il loro utilizzo nei problemi di classificazione.

Iniziamo con le importazioni standard:

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

#usiamo le impostazioni predefinite di plottaggio seaborn 
import seaborn as sns; sns.set_theme()

## Macchine vettoriali di supporto 
Come abbiamo già detto, le SVM fanno parte della classificazione discriminativa : anziché modellare ciascuna classe, troviamo semplicemente una linea o curva (in due dimensioni) o una varietà (in più dimensioni) che divide le classi l'una dall'altra.

Ad esempio, consideriamo il semplice caso di un compito di classificazione, in cui le due classi di punti sono ben separate:

In [ ]:
from sklearn.datasets import make_blobs
X, y = make_blobs(n_samples=50, centers=2,
                  random_state=0, cluster_std=0.60)
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn');

Come abbiamo detto, un classificatore discriminativo lineare tenterà di tracciare una linea retta che separa i due insiemi di dati e quindi di creare un modello per la classificazione. Per dati bidimensionali come quelli mostrati qui, questo è un compito che potremmo svolgere manualmente. Ma come detto in precedenza subito si presenterebbe un problema: esiste più di una possibile linea di demarcazione che possa discriminare perfettamente le due classi!

Andiamo a vederle:

In [ ]:
xfit = np.linspace(-1, 3.5)
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn')
plt.plot([0.6], [2.1], 'x', color='red', markeredgewidth=2, markersize=10)

for m, b in [(1, 0.65), (0.5, 1.6), (-0.2, 2.9)]:
    plt.plot(xfit, m * xfit + b, '-k')

plt.xlim(-1, 3.5)

Si tratta di tre separatori molto diversi che, tuttavia, discriminano perfettamente questi campioni. A seconda di quello che scegliamo, a un nuovo punto dati (ad esempio, quello contrassegnato dalla "X" in questo grafico) verrà assegnata un'etichetta diversa! Quindi evidentemente la nostra semplice intuizione di “tracciare una linea tra le classi” non è sufficiente, e dobbiamo pensare un po’ più a fondo.

## Macchine vettoriali di supporto: massimizzare margine
Le macchine vettoriali di supporto offrono appunto un modo per migliorare questo aspetto. Come detto l'intuizione è questa: invece di tracciare semplicemente una linea di larghezza zero tra le classi, possiamo tracciare attorno a ciascuna linea un margine di una certa larghezza, fino al punto più vicino. Ecco un esempio di come potrebbe apparire:

In [ ]:
xfit = np.linspace(-1, 3.5)
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn')

for m, b, d in [(1, 0.65, 0.33), (0.5, 1.6, 0.55), (-0.2, 2.9, 0.2)]:
    yfit = m * xfit + b
    plt.plot(xfit, yfit, '-k')
    plt.fill_between(xfit, yfit - d, yfit + d, edgecolor='none',
                     color='#AAAAAA', alpha=0.4)

plt.xlim(-1, 3.5)

Come già specificato nelle macchine a vettori di supporto, la linea che massimizza questo margine è quella che sceglieremo come modello ottimale. 

### Montaggio di una macchina vettoriale di supporto
Vediamo il risultato di un adattamento effettivo a questi dati: utilizzeremo il classificatore vettoriale di supporto di Scikit-Learn per addestrare un modello SVM su questi dati. Per il momento utilizzeremo un kernel lineare e imposteremo il parametro C su un numero molto grande:

In [ ]:
from sklearn.svm import SVC # "Support vector classifier"
model = SVC(kernel='linear', C=1E10)
model.fit(X, y)

Per visualizzare meglio cosa sta succedendo qui, creiamo una funzione rapida che traccerà per noi i limiti decisionali della SVM:

In [ ]:
def plot_svc_decision_function(model, ax=None, plot_support=True):
    """Creiamo la funzione decisionale per un SVC 2D""" 
    if ax is None:
        ax = plt.gca()
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    
    # create grid to evaluate model
    x = np.linspace(xlim[0], xlim[1], 30)
    y = np.linspace(ylim[0], ylim[1], 30)
    Y, X = np.meshgrid(y, x)
    xy = np.vstack([X.ravel(), Y.ravel()]).T
    P = model.decision_function(xy).reshape(X.shape)
    
    # plot decision boundary and margins
    ax.contour(X, Y, P, colors='k',
               levels=[-1, 0, 1], alpha=0.5,
               linestyles=['--', '-', '--'])
    
    # plot support vectors
    if plot_support:
        ax.scatter(model.support_vectors_[:, 0],
                   model.support_vectors_[:, 1],
                   s=300, linewidth=1, facecolors='none')
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

#creiamo il grafico
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn',edgecolors='black')
#richiamimamo la funzione
plot_svc_decision_function(model)

Questa è la linea di demarcazione che massimizza il margine tra i due insiemi di punti. Notiamo che alcuni punti di allenamento toccano appena il margine, questi punti sono gli elementi cardine di questo adattamento e sono noti come vettori di supporto e danno il nome all'algoritmo. In Scikit-Learn, l'identità di questi punti è memorizzata nell'attributo support_vectors_ del classificatore:

In [ ]:
print(model.support_vectors_)

Una chiave del successo di questo classificatore è che per l'adattamento conta solo la posizione dei vettori di supporto; eventuali punti più lontani dal margine che si trovano sul lato corretto non modificano l'adattamento! Tecnicamente, questo accade perché questi punti non contribuiscono alla funzione di perdita utilizzata per adattare il modello, quindi la loro posizione e il loro numero non contano finché non oltrepassano il margine.

Possiamo vederlo, a esempio, se tracciamo il modello appreso dai primi 60 punti e dai primi 120 punti di questo set di dati:

In [ ]:
def plot_svm(N=10, ax=None):
    X, y = make_blobs(n_samples=200, centers=2,
                      random_state=0, cluster_std=0.60)
    X = X[:N]
    y = y[:N]
    model = SVC(kernel='linear', C=1E10)
    model.fit(X, y)
    
    ax = ax or plt.gca()
    ax.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn', edgecolors='black')
    ax.set_xlim(-1, 4)
    ax.set_ylim(-1, 6)
    plot_svc_decision_function(model, ax)

fig, ax = plt.subplots(1, 2, figsize=(16, 6))
fig.subplots_adjust(left=0.0625, right=0.95, wspace=0.1)
for axi, N in zip(ax, [60, 120]):
    plot_svm(N, axi)
    axi.set_title('N = {0}'.format(N))

Nel pannello di sinistra vediamo il modello e i vettori di supporto per 60 punti di allenamento. Nel pannello di destra abbiamo raddoppiato il numero di punti di allenamento, ma il modello non è cambiato: i tre vettori di supporto del pannello di sinistra sono ancora i vettori di supporto del pannello di destra. Questa insensibilità al comportamento esatto dei punti distanti è uno dei punti di forza del modello SVM.

## Oltre i confini lineari: Kernel 
Il punto in cui SVM diventa estremamente potente è quando viene combinato con i kernel. 
Per motivare la necessità dei kernel, esaminiamo alcuni dati che non sono separabili linearmente:


In [ ]:
from sklearn.datasets import make_circles
X, y = make_circles(100, factor=.1, noise=.1)

clf = SVC(kernel='linear').fit(X, y)

plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn', edgecolors='black')
plot_svc_decision_function(clf, plot_support=False)

È chiaro che nessuna discriminazione lineare potrà mai separare questi dati, ma come già spiegato potremmo proiettare i dati in una dimensione più elevata in modo tale che un separatore lineare sia sufficiente. Ad esempio, una semplice proiezione che potremmo utilizzare sarebbe quella di calcolare una funzione di base radiale centrata sul gruppo centrale, per poi visualizzare questa dimensione di dati aggiuntiva utilizzando un grafico tridimensionale: 

In [ ]:
#funzione
r = np.exp(-(X ** 2).sum(1))

#grafico 3d
from mpl_toolkits import mplot3d

def plot_3D(elev=10, azim=30, X=X, y=y):
    ax = plt.subplot(projection='3d')
    ax.scatter3D(X[:, 0], X[:, 1], r, c=y, s=50, cmap='autumn',edgecolors='black')
    ax.view_init(elev=elev, azim=azim)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('r')

plot_3D() 


Possiamo vedere che con questa dimensione aggiuntiva, i dati diventano banalmente separabili linearmente, disegnando un piano di separazione, diciamo, a r = 0,7.

Qui abbiamo dovuto scegliere e mettere a punto con attenzione la nostra proiezione: se non avessimo centrato la nostra funzione di base radiale nella posizione giusta, non avremmo visto risultati così netti e linearmente separabili. In generale, la necessità di fare una scelta del genere è un problema: vorremmo in qualche modo trovare automaticamente le migliori funzioni di base da utilizzare.

Una strategia a tal fine è calcolare una funzione di base centrata in ogni punto del set di dati e lasciare che l'algoritmo SVM esamini i risultati. Questo tipo di trasformazione della funzione di base è nota come trasformazione del kernel , poiché si basa su una relazione di somiglianza (o kernel) tra ciascuna coppia di punti.

Può sorgere un potenziale problema con questa strategia: la proiezione N punti in N dimensioni potrebbe diventare molto intensivo dal punto di vista computazionale man mano che N diventa grande. Tuttavia, grazie a una piccola procedura nota come trucco del kernel, può essere eseguito implicitamente un adattamento sui dati trasformati dal kernel , ovvero senza mai creare l'intero file.
Questo trucco del kernel è integrato nell'SVM ed è uno dei motivi per cui il metodo è così potente.

In Scikit-Learn, possiamo applicare SVM kernelizzato semplicemente cambiando il nostro kernel lineare in un kernel RBF (funzione a base radiale), utilizzando il paramentro kernel del modello:

In [ ]:
clf = SVC(kernel='rbf', C=1E6)
clf.fit(X, y)


plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn',edgecolors='black')
plot_svc_decision_function(clf)
plt.scatter(clf.support_vectors_[:, 0], clf.support_vectors_[:, 1],
            s=300, lw=1, facecolors='none')

Usando questa macchina vettoriale di supporto kernelizzato, impariamo un opportuno confine decisionale non lineare. Questa strategia di trasformazione del kernel viene utilizzata spesso nell'apprendimento automatico per trasformare metodi lineari veloci in metodi non lineari veloci, in particolare per i modelli in cui è possibile utilizzare il trucco del kernel.

### Ottimizzazione dell'SVM: Attenuazione
La nostra discussione finora si è incentrata su set di dati molto puliti, in cui esiste un confine decisionale perfetto. Ma cosa succede se i nostri dati presentano una certa sovrapposizione? A esempio, potremmo avere dati come questi:

In [ ]:
X, y = make_blobs(n_samples=100, centers=2,
                  random_state=0, cluster_std=1.2)
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn',edgecolors='black')

Per gestire questo caso, l'implementazione SVM come abbiamo detto ha un fattore di confusione che "ammorbidisce" il margine: ovvero consente ad alcuni punti di insinuarsi nel margine se ciò consente un adattamento migliore. La durezza del margine è controllata da un parametro di regolazione, spesso noto come C. Per C molto grande, il margine è duro e i punti non possono trovarsi al suo interno. Per C più piccoli, il margine è più morbido e può crescere fino a comprendere alcuni punti.

La trama mostrata di seguito fornisce un'immagine visiva di come avviene un cambiamento C , il parametro influisce sull'adattamento finale, tramite l'ammorbidimento del margine:

In [ ]:
X, y = make_blobs(n_samples=100, centers=2,
                  random_state=0, cluster_std=0.8)

fig, ax = plt.subplots(1, 2, figsize=(16, 6))
fig.subplots_adjust(left=0.0625, right=0.95, wspace=0.1)

for axi, C in zip(ax, [10.0, 0.1]):
    model = SVC(kernel='linear', C=C).fit(X, y)
    axi.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn',edgecolors='black')
    plot_svc_decision_function(model, axi)
    axi.scatter(model.support_vectors_[:, 0],
                model.support_vectors_[:, 1],
                s=300, lw=1, facecolors='none')
    axi.set_title('C = {0:.1f}'.format(C), size=14)

### Andiamo ora a vedere come può essere applicato l'algoritmo SVM con lo stesso problema già visto del riconoscimento di cifre scritte a mano
Prima cosa importiamo le librerie:

In [ ]:
import matplotlib.pyplot as plt

# Import datasets, classifiers and performance metrics
from sklearn import datasets, metrics, svm
from sklearn.model_selection import train_test_split

Vi ricordo che il set di dati delle cifre è costituito da immagini di cifre da 8x8 pixel. L' images attributo del set di dati memorizza matrici 8x8 di valori in scala di grigi per ciascuna immagine. Utilizzeremo questi array per visualizzare le prime 4 immagini. L'attributo target del set di dati memorizza la cifra rappresentata da ciascuna immagine e questa è inclusa nel titolo dei 4 grafici seguenti:

In [ ]:
digits = datasets.load_digits()

_, axes = plt.subplots(nrows=1, ncols=4, figsize=(10, 3))
for ax, image, label in zip(axes, digits.images, digits.target):
    ax.set_axis_off()
    ax.imshow(image, cmap=plt.cm.gray_r, interpolation="nearest")
    ax.set_title("Training: %i" % label)

Per applicare il classificatore a questi dati, procediamo ad appiattire le immagini, trasformando ogni matrice 2D di valori in scala di grigio da forma(8, 8) a forma (64,).

Possiamo quindi suddividere i dati in sottoinsiemi di allenamento e test e inserire un classificatore di vettori di supporto sui campioni del train. Il classificatore adattato può successivamente essere utilizzato per prevedere il valore della cifra per i campioni nel sottoinsieme del test.

In [ ]:
# appiattiamo le immagini
n_samples = len(digits.images)
data = digits.images.reshape((n_samples, -1))

# Creiamo il classificatore
clf = svm.SVC(gamma=0.001)

# Dividiamo i dati per training e test al 50%
X_train, X_test, y_train, y_test = train_test_split(
    data, digits.target, test_size=0.5, shuffle=False
)

# alleniamo il modello
clf.fit(X_train, y_train)

# Prevediamo i valori
predicted = clf.predict(X_test)

Andiamo quindi a visualizzare i primi 4 campioni di prova e mostriamo il loro valore numerico di previsione nel titolo:

In [ ]:
_, axes = plt.subplots(nrows=1, ncols=4, figsize=(10, 3))
for ax, image, prediction in zip(axes, X_test, predicted):
    ax.set_axis_off()
    image = image.reshape(8, 8)
    ax.imshow(image, cmap=plt.cm.gray_r, interpolation="nearest")
    ax.set_title(f"Prediction: {prediction}")

Il risultato sembra abbastanza buono ma avendo cifre molto difficili da catalogare anche a occhio umano, ci conviene prima stampare un report:

In [ ]:
print(
    f"Classification report for classifier {clf}:\n"
    f"{metrics.classification_report(y_test, predicted)}\n"
)

E poi tracciare una matrice di confusione dei valori delle cifre reali e dei valori delle cifre previste:

In [ ]:
disp = metrics.ConfusionMatrixDisplay.from_predictions(y_test, predicted)
disp.figure_.suptitle("Confusion Matrix")
print(f"Confusion matrix:\n{disp.confusion_matrix}")
plt.show()

In [ ]:

#finora avevamo visto quest'altro metodo per creare grafici che richiedeva più passaggi:

cm = metrics.confusion_matrix(y_test, predicted, labels=clf.classes_)
disp = metrics.ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=clf.classes_)
disp.plot()

plt.show()

Andiamo ora a vedere al gli altri kernel utilizzabili.
- Prima cosa creiamo un set di dati di classificazione bidimensionale con 16 campioni e due classi e tracciamo i campioni con i colori corrispondenti ai rispettivi target.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

X = np.array(
    [
        [0.4, -0.7],
        [-1.5, -1.0],
        [-1.4, -0.9],
        [-1.3, -1.2],
        [-1.1, -0.2],
        [-1.2, -0.4],
        [-0.5, 1.2],
        [-1.5, 2.1],
        [1.0, 1.0],
        [1.3, 0.8],
        [1.2, 0.5],
        [0.2, -2.0],
        [0.5, -2.4],
        [0.2, -2.3],
        [0.0, -2.7],
        [1.3, 2.1],
    ]
)

y = np.array([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1])

# Plotting settings
fig, ax = plt.subplots(figsize=(4, 3))
x_min, x_max, y_min, y_max = -3, 3, -3, 3
ax.set(xlim=(x_min, x_max), ylim=(y_min, y_max))

# Plot samples by color and add legend
scatter = ax.scatter(X[:, 0], X[:, 1], s=150, c=y, label=y, edgecolors="k")
ax.legend(*scatter.legend_elements(), loc="upper right", title="Classes")
ax.set_title("Samples in two-dimensional feature space")
_ = plt.show()

Possiamo vedere che i campioni non sono chiaramente separabili da una linea retta.
Definiamo  a questo punto una funzione che si adatta a un SVM, consentendo il parametro kernel come input, quindi tracciamo i confini decisionali appresi dal modello utilizzando DecisionBoundaryDisplay.

Per semplicità, in questo esempio il parametro C è impostato sul suo valore predefinito ( C=1) su tutti i kernel e la gamma=2. In un'attività di classificazione reale, in cui le prestazioni contano, è altamente consigliabile l'ottimizzazione dei parametri per acquisire diverse strutture all'interno dei dati.

Impostiamo response_method="predict" dei colori DecisionBoundaryDisplay in base alle aree della classe prevista. L'utilizzo response_method="decision_function" ci consente anche di tracciare il confine della decisione e i margini su entrambi i lati di esso. Infine i vettori di supporto utilizzati durante l'addestramento (che giacciono sempre ai margini) vengono identificati mediante l'attributo support_vectors_ degli SVM addestrati e anche tracciati.



In [ ]:
from sklearn import svm
from sklearn.inspection import DecisionBoundaryDisplay


def plot_training_data_with_decision_boundary(kernel):
    # Train the SVC
    clf = svm.SVC(kernel=kernel, gamma=2).fit(X, y)

    # Settings for plotting
    _, ax = plt.subplots(figsize=(4, 3))
    x_min, x_max, y_min, y_max = -3, 3, -3, 3
    ax.set(xlim=(x_min, x_max), ylim=(y_min, y_max))

    # Plot decision boundary and margins
    common_params = {"estimator": clf, "X": X, "ax": ax}
    DecisionBoundaryDisplay.from_estimator(
        **common_params,
        response_method="predict",
        plot_method="pcolormesh",
        alpha=0.3,
    )
    DecisionBoundaryDisplay.from_estimator(
        **common_params,
        response_method="decision_function",
        plot_method="contour",
        levels=[-1, 0, 1],
        colors=["k", "k", "k"],
        linestyles=["--", "-", "--"],
    )

    # Plot bigger circles around samples that serve as support vectors
    ax.scatter(
        clf.support_vectors_[:, 0],
        clf.support_vectors_[:, 1],
        s=250,
        facecolors="none",
        edgecolors="k",
    )
    # Plot samples by color and add legend
    ax.scatter(X[:, 0], X[:, 1], c=y, s=150, edgecolors="k")
    ax.legend(*scatter.legend_elements(), loc="upper right", title="Classes")
    ax.set_title(f" Decision boundaries of {kernel} kernel in SVC")

    _ = plt.show()

Rivediamo il kernel lineare nel quale l'addestramento produce uno spazio di caratteristiche non trasformato, dove l'iperpiano e i margini sono linee rette. A causa della mancanza di espressività del kernel lineare, le classi addestrate non catturano perfettamente i dati di addestramento.



In [ ]:
plot_training_data_with_decision_boundary("linear")

### Ora vediamo il kernel polinomiale in cui la nozione di somiglianza è smussata:

Qui utilizziamo il valore predefinito per il grado del polinomio nella funzione kernel (degree=3). Quando coef0=0 (impostazione predefinita), i dati vengono solo trasformati, ma non viene aggiunta alcuna dimensione aggiuntiva. Usare un kernel polinomiale equivale a creare PolynomialFeatures e quindi adattare un kernel lineare ai dati trasformati, sebbene questo approccio alternativo sarebbe costoso dal punto di vista computazionale per la maggior parte dei set di dati. 
Come vediamo il nucleo polinomiale gamma=2 si adatta bene ai dati di addestramento, facendo piegare di conseguenza i margini su entrambi i lati dell'iperpiano.



In [ ]:
plot_training_data_with_decision_boundary("poly")

### Poi abbiamo il kernel RBF (Radial Basis Function), noto anche come kernel gaussiano.

E' il kernel predefinito per Support Vector Machines in scikit-learn, come sappiamo misura la somiglianza tra due punti dati in dimensioni infinite e quindi si avvicina alla classificazione in base al voto a maggioranza.
Nel grafico possiamo vedere come i confini decisionali tendono a contrarsi attorno a punti dati vicini tra loro.

In [ ]:
plot_training_data_with_decision_boundary("rbf")

### Sigmoid kernel
Possiamo vedere che i confini decisionali ottenuti con il nucleo sigmoideo appaiono curvi e irregolari. Il confine decisionale tenta di separare le classi adattando una curva a forma di sigmoide, risultando in un confine complesso che potrebbe non generalizzarsi bene ai dati invisibili. Da questo esempio diventa ovvio che il kernel sigmoideo ha casi d'uso molto specifici, quando si tratta di dati che presentano una forma sigmoidale. In questo esempio, un’attenta messa a punto potrebbe trovare limiti decisionali più generalizzabili. A causa della sua specificità, il kernel sigmoideo è usato meno comunemente nella pratica rispetto ad altri kernel.

In [ ]:
plot_training_data_with_decision_boundary("sigmoid")

## Esercizio 1

L'obiettivo di questo esercizio è identificare il kernel migliore dell'algoritmo SVM per classificare il tipo di fiore "setosa" e "virginica" del dataset iris.
Di seguito la base per importare il dataset e le classi specifiche

In [ ]:
from sklearn import datasets

iris = datasets.load_iris()
X = iris.data
y = iris.target

X = X[y != 1, :2]
y = y[y != 1]

#grafico per vedere il dataset nella totalità
import matplotlib.pyplot as plt

_, ax = plt.subplots()
scatter = ax.scatter(iris.data[:, 0], iris.data[:, 1], c=iris.target)
ax.set(xlabel=iris.feature_names[0], ylabel=iris.feature_names[1])
_ = ax.legend(
    scatter.legend_elements()[0], iris.target_names, loc="lower right", title="Classes"
)

## Esempio reale di utilizzo di SVM per Riconoscimento Facciale
Come esempio di macchine vettoriali di supporto in azione, diamo un'occhiata al problema del riconoscimento facciale, utilizzeremo il set di dati Labeled Faces in the Wild, che consiste di diverse migliaia di foto raccolte di vari personaggi pubblici che è già incluso in Scikit-Learn:

In [ ]:
from sklearn.datasets import fetch_lfw_people
import matplotlib.pyplot as plt
faces = fetch_lfw_people(min_faces_per_person=60)

#alcune delle immagini presenti
print(faces.target_names[0])
print(faces.target_names[1])
print(faces.target_names[2])

#dimensioni dataset
print(faces.images.shape)

Andiamo ora a vedere alcuni di questi volti per capire con cosa stiamo lavorando:

In [ ]:
fig, ax = plt.subplots(3, 4)
for i, axi in enumerate(ax.flat):
    axi.imshow(faces.images[i], cmap='bone')
    axi.set(xticks=[], yticks=[],
            xlabel=faces.target_names[faces.target[i]])

Ogni immagine contiene [62×47] quasi 3.000 pixel. Potremmo procedere semplicemente utilizzando ciascun valore di pixel come caratteristica, ma spesso è più efficace utilizzare una sorta di preprocessore per estrarre caratteristiche più significative; qui utilizzeremo un'analisi delle componenti principali per estrarre 150 componenti fondamentali da inserire nel nostro classificatore di macchine vettoriali di supporto. 
Possiamo farlo nel modo più semplice impacchettando il preprocessore e il classificatore in un'unica pipeline:

In [ ]:
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline

pca = PCA(n_components=150, whiten=True, random_state=42)
svc = SVC(kernel='rbf', class_weight='balanced')
model = make_pipeline(pca, svc)

Per testare l'output del nostro classificatore, divideremo i dati in un set in training e test:

In [ ]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(faces.data, faces.target,
                                                random_state=42)

Infine, possiamo utilizzare una convalida incrociata della ricerca sulla griglia per esplorare combinazioni di parametri. Qui regoleremo C (che come sappiamo controlla la durezza del margine) e gamma (che controlla la dimensione del nucleo della funzione base radiale) e determineremo il modello migliore:

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {'svc__C': [1, 5, 10, 50],
              'svc__gamma': [0.0001, 0.0005, 0.001, 0.005]}
grid = GridSearchCV(model, param_grid, n_jobs=-1)

%time grid.fit(Xtrain, ytrain)
print(grid.best_params_)

I valori ottimali si trovano verso la metà della nostra griglia, se si trovassero ai bordi, dovremmo espandere la griglia per essere sicuri di aver trovato il vero ottimale.

Ora, con questo modello a convalida incrociata, possiamo prevedere le etichette per i dati di test, che il modello non ha ancora visto:

In [ ]:
model = grid.best_estimator_
yfit = model.predict(Xtest)

Diamo un'occhiata ad alcune delle immagini di prova insieme ai loro valori previsti:

In [ ]:
fig, ax = plt.subplots(4, 6)
for i, axi in enumerate(ax.flat):
    axi.imshow(Xtest[i].reshape(62, 47), cmap='bone')
    axi.set(xticks=[], yticks=[])
    axi.set_ylabel(faces.target_names[yfit[i]].split()[-1],
                   color='black' if yfit[i] == ytest[i] else 'red')
fig.suptitle('Predicted Names; Incorrect Labels in Red', size=14)

Di questo piccolo campione, il nostro stimatore ottimale ha etichettato erroneamente solo due facce. Possiamo avere un'idea migliore delle prestazioni del nostro stimatore utilizzando il rapporto di classificazione, che elenca le statistiche di recupero etichetta per etichetta:

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(ytest, yfit,
                            target_names=faces.target_names))

Potremmo anche visualizzare la matrice di confusione tra queste classi:

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
mat = confusion_matrix(ytest, yfit)
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
            xticklabels=faces.target_names,
            yticklabels=faces.target_names)
plt.xlabel('true label')
plt.ylabel('predicted label')

Questo ci aiuta a capire quali etichette potrebbero essere confuse dallo stimatore.

Per un'attività di riconoscimento facciale nel mondo reale, in cui le foto non vengono preritagliate in belle griglie, l'unica differenza nello schema di classificazione facciale è la selezione delle funzionalità: dovremmo utilizzare un algoritmo più sofisticato per trovare i volti ed estrarre funzionalità indipendenti dalla pixellazione. 

## Esercizio 2
Utilizzate il dataset "Wine" disponibile nella libreria scikit-learn. L'obiettivo sarà classificare il tipo di vino in base alle sue caratteristiche chimiche.

### Passaggi da fare:

- Caricamento dei dati:

1. Carica il dataset "Wine" dalla libreria scikit-learn.
2. Esplora il dataset per comprendere le caratteristiche presenti, i loro tipi e la distribuzione delle classi di output.

- Preprocessing dei dati:

1. Dividi il dataset in features (variabili indipendenti) e target (variabile dipendente).
2. Dividi il dataset in training set e test set utilizzando una proporzione del 80-20.

- Standardizzazione dei dati:

1. Standardizza le features utilizzando lo StandardScaler di scikit-learn.

- Creazione del modello SVM:

1. Crea un modello SVM utilizzando il kernel lineare.

- Addestramento del modello:

1. Addestra il modello SVM sul training set.

- Valutazione del modello:

1. Valuta le prestazioni del modello utilizzando il test set.
2. Calcola l'accuratezza del modello.
3. Visualizza il report di classificazione che include precision, recall e F1-score per ogni classe.
3. Visualizza la matrice di confusione per valutare le prestazioni del modello.


